# Finding ORFs
Autor: Luiz Carlos Vieira

### In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5').

An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.  

The following program identify all ORFs present on the forward strand (1, 2, or 3) in each sequence of the FASTA file.  

In [1]:
# import modules
from Bio import SeqIO
import pandas as pd

In [2]:
# to read single fasta file
# seq_obj = SeqIO.read("dna.fasta", "fasta")

# to read multi fasta file
seq_obj = SeqIO.parse("dna.fasta", "fasta")

sequences=[]
for seq in seq_obj:
    sequences.append(seq)

In [3]:
# get total number of records in the multi-fasta file
len(sequences)

25

In [4]:
# get the first record
record1 =  sequences[0]
record1

SeqRecord(seq=Seq('TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGCGGGCCTCT...TTC'), id='gi|142022655|gb|EQ086233.1|43', name='gi|142022655|gb|EQ086233.1|43', description='gi|142022655|gb|EQ086233.1|43 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[])

In [5]:
# ID of first record
print(record1.id)

gi|142022655|gb|EQ086233.1|43


In [6]:
# name of first record
print(record1.name)

gi|142022655|gb|EQ086233.1|43


In [7]:
# description of first record
print(record1.description)

gi|142022655|gb|EQ086233.1|43 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence


In [8]:
# loop to get record ids and sequence length
seq_ids = []
seq_lengths = []

for seq in sequences:
    seq_id=seq.id
    sequence=seq.seq
    length=len(sequence)
    
    seq_ids.append(seq_id)
    seq_lengths.append(length)

In [9]:
# Create a dataframe and store the sequence ids and lengths
df = pd.DataFrame()
df['Seq_Id']=seq_ids
df['Seq_Length']=seq_lengths
df

,Seq_Id,Seq_Length
0,gi|142022655|gb|EQ086233.1|43,990
1,gi|142022655|gb|EQ086233.1|160,724
2,gi|142022655|gb|EQ086233.1|41,3080
3,gi|142022655|gb|EQ086233.1|221,2863
4,gi|142022655|gb|EQ086233.1|294,3832
5,gi|142022655|gb|EQ086233.1|323,4805
6,gi|142022655|gb|EQ086233.1|564,1663
7,gi|142022655|gb|EQ086233.1|521,512
8,gi|142022655|gb|EQ086233.1|455,691
9,gi|142022655|gb|EQ086233.1|229,3072


In [10]:
# ORF FINDER

# Function to find all frames
def frames(seq, frame):
    frame1 = seq[0:len(seq)]
    frame2 = seq[1:len(seq)]
    frame3 = seq[2:len(seq)]
    if frame == 1:
        return(frame1)
    elif frame == 2:
        return(frame2)
    elif frame == 3:
        return(frame3)


# Function to find all start codons
def find_starts(seq):
    starts = []
    for i in range(0, len(seq), 3):
        start_codon = seq[i:i+3]
        if start_codon == "ATG":
            idx1 = i
            starts.append(idx1)
    starts = sorted(starts)
    return(starts)


# function to find a stop codons
def find_stop(start, seq):
    stops = []
    for j in range(start, len(seq), 3):
        stop_codon = seq[j:j+3]
        if stop_codon not in ("TAG","TAA","TGA"):
            stop = 0
        if stop_codon in ("TAG","TAA","TGA"):
            stop = j + 3
            break
    stops.append(stop)
    return(stops[0])


# Function to find orfs
def ORF_finder(sequence, frame):
    seq = frames(sequence, frame)
    start = find_starts(seq)
    res = []
    for i in start:
        j = find_stop(i, seq)
        if j != 0:
            orf = seq[i:j]
            l = len(orf)
            res.append([orf, i, j, l])
    return(res)

# Function to find the long orf
def long_ORFs(sequence, frame):
    orfs = []
    a = ORF_finder(sequence, frame)
    for j in range(0, len(a)):
        b = a[j][3]
        orfs.append(b)
    orfs = sorted(orfs, reverse=True)
    return(orfs)

# Function to find the long orf
def long_ORF(sequence, frame):
    #orfs = {}
    orfs = []
    a = ORF_finder(sequence, frame)
    long = 0
    for j in range(0, len(a)):
        if a[j][3] > long:
            long = a[j][3]
            data = a[j][0:4]        
        #orfs.update({'frame'+str(frame):data})
    orfs.append(['frame'+str(frame),data])
    return(orfs)

# Function to find the longests orfs
def longest_ORFs(sequence, n):
    orfs = []
    for i in range(1, 4):
        a = ORF_finder(sequence, i)
        for j in range(0, len(a)):
            b = a[j][3]
            orfs.append(b)
    orfs = sorted(orfs, reverse=True)
    return(orfs[0:n])


## Testing the the ORF_finder

In [11]:
# Creating a dictionary to store the orfs found
dict_ = {}
for seq in sequences:
    a = longest_ORFs(seq.seq, 1)
    dict_.update({seq.id:a})

dict_

{'gi|142022655|gb|EQ086233.1|43': [213],
 'gi|142022655|gb|EQ086233.1|160': [363],
 'gi|142022655|gb|EQ086233.1|41': [918],
 'gi|142022655|gb|EQ086233.1|221': [594],
 'gi|142022655|gb|EQ086233.1|294': [1608],
 'gi|142022655|gb|EQ086233.1|323': [1686],
 'gi|142022655|gb|EQ086233.1|564': [507],
 'gi|142022655|gb|EQ086233.1|521': [159],
 'gi|142022655|gb|EQ086233.1|455': [552],
 'gi|142022655|gb|EQ086233.1|229': [1311],
 'gi|142022655|gb|EQ086233.1|422': [639],
 'gi|142022655|gb|EQ086233.1|384': [720],
 'gi|142022655|gb|EQ086233.1|280': [291],
 'gi|142022655|gb|EQ086233.1|158': [1218],
 'gi|142022655|gb|EQ086233.1|59': [1107],
 'gi|142022655|gb|EQ086233.1|319': [537],
 'gi|142022655|gb|EQ086233.1|438': [477],
 'gi|142022655|gb|EQ086233.1|210': [564],
 'gi|142022655|gb|EQ086233.1|237': [219],
 'gi|142022655|gb|EQ086233.1|507': [741],
 'gi|142022655|gb|EQ086233.1|350': [678],
 'gi|142022655|gb|EQ086233.1|245': [636],
 'gi|142022655|gb|EQ086233.1|279': [153],
 'gi|142022655|gb|EQ086233.1|378

In [12]:
# Create a new column to df, longest orf
df["longest_orf"] = dict_.values()
df

,Seq_Id,Seq_Length,longest_orf
0,gi|142022655|gb|EQ086233.1|43,990,[213]
1,gi|142022655|gb|EQ086233.1|160,724,[363]
2,gi|142022655|gb|EQ086233.1|41,3080,[918]
3,gi|142022655|gb|EQ086233.1|221,2863,[594]
4,gi|142022655|gb|EQ086233.1|294,3832,[1608]
5,gi|142022655|gb|EQ086233.1|323,4805,[1686]
6,gi|142022655|gb|EQ086233.1|564,1663,[507]
7,gi|142022655|gb|EQ086233.1|521,512,[159]
8,gi|142022655|gb|EQ086233.1|455,691,[552]
9,gi|142022655|gb|EQ086233.1|229,3072,[1311]


In [13]:
# Getting the se with the longest orf 
record5 = sequences[5]
record5

SeqRecord(seq=Seq('ACGCCCGGCGCACCGCGAGTACCGCGCCGCCGGGCACTCCTTGACCCCGCATGA...CGC'), id='gi|142022655|gb|EQ086233.1|323', name='gi|142022655|gb|EQ086233.1|323', description='gi|142022655|gb|EQ086233.1|323 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[])

In [14]:
large_orfs = []
for i in range(1,4):
    a = long_ORF(record5.seq, i)
    #print(a[0][1][0])
    large_orfs.append(a[0][1][0])

large_orfs

[Seq('ATGCGAACCAGTTGTTCGGACACCTCGACGAGCAAGTCGTGCCGCAGGCGCGCG...TAG'),
 Seq('ATGAAACCCGAAAACCTCGTCGCCTGCCACGAATGCGACCTGCTGTTTTGGCGG...TGA'),
 Seq('ATGCTTCCCCTCGATTTGCCTGAACCCGAGATCCGGCCGCGCAGCCGCTGGATC...TGA')]

In [15]:
prot_list = []
for orfs in large_orfs:
    trans = orfs.translate(1)
    trans_len = len(trans)
    prot_list.append((trans, trans_len))

prot_list

[(Seq('MRTSCSDTSTSKSCRRRATRWRRRNARSTPRRRRCGRIRRCNRTFMTRCNRSRR...PA*'), 562),
 (Seq('MKPENLVACHECDLLFWRPPRLRALAAHCPRCRARVGGSAHGRPALDRRCAIAL...RS*'), 457),
 (Seq('MLPLDLPEPEIRPRSRWIPSLVWIVPLVCALIGLALVYRGIAATGPTITVTFAN...PK*'), 532)]

In [16]:
## Importing modules to do the blast
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

In [17]:
# Realizing a protein blast
blast_handle = NCBIWWW.qblast('blastp', 'pdb', prot_list[1])

# Saving blast results to a variable blast_record
blast_record = NCBIXML.read(blast_handle)

# Visualizing the results of blast
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        print('******************************************Alignment******************************************')
        print('sequence:', alignment.title)
        print('length:', alignment.length)
        print('e value:', hsp.expect)
        print(hsp.query)
        print(hsp.match)
        print(hsp.sbjct)
        print()

******************************************Alignment******************************************
sequence: pdb|6QB7|A Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|B Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|C Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|D Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|E Structure of the H1 domain of human KCTD16 [Homo sapiens]
length: 163
e value: 4.95091
TPRRRRCGRIRRCNRTFMTRCNRSRRP
 PR   CGRI      F    N SR P
VPRILVCGRISLAKEVFGETLNESRDP



## Defining a function to blast

In [18]:
def blast(blast_type, datadabe, proten_sequence):
    # Realizing a protein blast
    blast_handle = NCBIWWW.qblast(blast_type, datadabe, proten_sequence)

    # Saving blast results to a variable blast_record
    blast_record = NCBIXML.read(blast_handle)

    # Visualizing the results of blast
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print('******************************************Alignment******************************************')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query)
            print(hsp.match)
            print(hsp.sbjct)
            print()

In [19]:
# protein blast to the protein 2 in the list
blast('blastp', 'swissprot', prot_list[1])

******************************************Alignment******************************************
sequence: sp|Q6GMD3.1| RecName: Full=Shiftless antiviral inhibitor of ribosomal frameshifting protein homolog; Short=SHFL; AltName: Full=Repressor of yield of DENV protein homolog [Xenopus laevis]
length: 305
e value: 0.378018
ACHECDLLFWRPPRLRALAAHCPRCRAR
AC ECD ++WR    R   + C RCR +
ACKECDYMWWRRVPQRKEVSRCQRCRKK

******************************************Alignment******************************************
sequence: sp|O55005.1| RecName: Full=Roundabout homolog 1; Flags: Precursor [Rattus norvegicus]
length: 1651
e value: 0.392471
PRLRALAAHCPRCRARVGGSAHGRPALDRRCAIALRS
P+L ++ A   R   R GGS  GR ALD R    LR+
PKLASIEARADRSSDRKGGSYKGREALDGRQVTDLRT

******************************************Alignment******************************************
sequence: sp|O89026.1| RecName: Full=Roundabout homolog 1; Flags: Precursor [Mus musculus]
length: 1612
e value: 0.502584
PRLRALAAHCPRCRARVGGSAHGRPALDRRCAIALRS
P

In [20]:
# For loop to blast all sequences from a list of protein sequence
for i in range(0, len(prot_list)):
    blast("blastp", "pdb", prot_list[i])

******************************************Alignment******************************************
sequence: pdb|6QB7|A Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|B Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|C Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|D Structure of the H1 domain of human KCTD16 [Homo sapiens] >pdb|6QB7|E Structure of the H1 domain of human KCTD16 [Homo sapiens]
length: 163
e value: 4.95091
TPRRRRCGRIRRCNRTFMTRCNRSRRP
 PR   CGRI      F    N SR P
VPRILVCGRISLAKEVFGETLNESRDP

******************************************Alignment******************************************
sequence: pdb|7ANE|at Chain at, mS66 [Leishmania major]
length: 397
e value: 1.2372
PRLRALAAHCPRCRARVGGSAHGRPALDRRCAIALRS
PR+   +AHCP C  R   +A GR A +    + L +
PRITEWSAHCPACAWRTNMTAIGRKAQEEGQYLGLET

******************************************Alignment******************************************
sequence: pdb|7D5K|A Chain A, Cellulose syntha